\
            # 37. 进程（multiprocessing）（Multiprocessing）

            多进程适合 CPU 密集并行计算：绕开 GIL、利用多核。
本节强调 Windows/Jupyter 下的 spawn 注意事项，并给出“写脚本再运行”的稳定模板。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 并发基础
- 函数/异常
- 了解 __main__ 入口保护


## 知识点地图

- 1. 为什么要多进程：绕开 GIL 做 CPU 并行
- 2. Windows/Jupyter 的关键点：spawn + __main__
- 3. 进程池（推荐）：ProcessPoolExecutor
- 4. 跨进程通信：Queue/Pipe/共享对象（了解）
- 5. pickle 限制：哪些东西不能传给子进程


## 自检清单（学完打勾）

- [ ] 知道何时用进程（CPU 密集）以及代价（序列化开销）
- [ ] 理解 Windows 的 spawn 与 __main__ 保护
- [ ] 会用进程池并能拿到子进程异常
- [ ] 知道跨进程通信需要 pickle（可序列化）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：为什么要多进程：绕开 GIL 做 CPU 并行

- GIL 让同一时刻只有一个线程执行 Python 字节码。
- CPU 密集任务：多线程往往无法线性加速；多进程可以利用多核。
- 代价：创建/切换更重；数据要序列化（pickle）；进程间共享内存更麻烦。

经验：尽量让任务“输入小、输出小”，在主进程汇总结果。


## 知识点 2：Windows/Jupyter 的关键点：spawn + __main__

- Windows 默认启动方式是 **spawn**：新进程会重新导入主模块。
- 必须用 `if __name__ == "__main__":` 保护创建子进程的代码。
- 在 Jupyter/交互环境里直接起多进程可能受启动方式影响；更稳的是“写脚本再运行”。


## 知识点 3：进程池（推荐）：ProcessPoolExecutor

- `concurrent.futures.ProcessPoolExecutor` 是更现代的接口：与线程池用法一致。
- 适合“纯函数式”CPU 任务：输入/输出可 pickle。
- 子进程异常会通过 `Future.result()` 传回主进程。

下面示例在 Notebook 里可能不稳定，因此只展示检测逻辑与写法。


In [ ]:
import sys
from concurrent.futures import ProcessPoolExecutor


def cpu_work(n: int) -> int:
    s = 0
    for i in range(n):
        s += (i * i) % 97
    return s


in_notebook = 'ipykernel' in sys.modules
print('in_notebook:', in_notebook)

if not in_notebook:
    with ProcessPoolExecutor(max_workers=2) as ex:
        print(list(ex.map(cpu_work, [300_000, 300_000, 300_000, 300_000])))
else:
    print('建议用本章小案例：把多进程写入脚本再运行，更稳定。')


## 知识点 4：跨进程通信：Queue/Pipe/共享对象（了解）

- `multiprocessing.Queue`：进程安全队列，底层管道 + 序列化。
- `Pipe`：双端通信，适合点对点。
- `Value/Array`：共享内存的基础类型；`Manager()` 可共享 dict/list（更慢）。

规则：共享越多越慢；能“分而治之”就别共享。


## 知识点 5：pickle 限制：哪些东西不能传给子进程

常见不可 pickle：
- lambda、局部函数、闭包中捕获了不可序列化对象的函数
- 打开的文件句柄、socket、线程锁

建议：把 worker 函数定义在模块顶层；传入简单数据结构（数字/字符串/小 dict）。


## 常见坑

- 在 Windows/Jupyter 里直接开进程却没有 __main__ 保护
- 给子进程传 lambda/局部函数：pickle 失败
- 让子进程频繁传大对象：序列化开销吞掉收益
- 忽略回收：子进程里忘记关闭文件/连接


## 综合小案例：写脚本跑多进程：并行统计素数个数

这个案例会把脚本写入 `_nb_artifacts/mp_primes.py` 并用 `subprocess` 运行。

目的：在 Windows 的 Notebook 环境里保证可复现、可稳定运行。


In [ ]:
import subprocess
import sys
from pathlib import Path

script = r"""
from __future__ import annotations

import math
from concurrent.futures import ProcessPoolExecutor


def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    r = int(math.isqrt(n))
    for i in range(3, r + 1, 2):
        if n % i == 0:
            return False
    return True


def count_primes(lo: int, hi: int) -> int:
    return sum(1 for x in range(lo, hi) if is_prime(x))


def count_range(r):
    lo, hi = r
    return count_primes(lo, hi)


def main():
    ranges = [(10_000, 20_000), (20_000, 30_000), (30_000, 40_000), (40_000, 50_000)]
    with ProcessPoolExecutor(max_workers=4) as ex:
        counts = list(ex.map(count_range, ranges))
    print('ranges:', ranges)
    print('counts:', counts)
    print('total:', sum(counts))


if __name__ == '__main__':
    main()
"""

path = Path('_nb_artifacts') / 'mp_primes.py'
path.write_text(script, encoding='utf-8')
print('wrote', path)

out = subprocess.check_output([sys.executable, str(path)], text=True, encoding='utf-8')
print(out)


## 自测题（不写代码也能回答）

- 为什么 Windows 默认是 spawn？这对程序结构有什么要求？
- 子进程为什么需要 pickle？哪些对象常见不可 pickle？
- 多进程什么时候反而更慢？


## 练习题（建议写代码）

- 把小案例改成：动态任务队列（让 worker 从队列取 range），提升负载均衡。
- 用 `multiprocessing.Queue` 实现“子进程产出结果，主进程汇总”。
- 尝试将计算拆到更小任务粒度，比较开销与速度（写下结论）。
